In [ ]:
!pip install GPUtil

import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU not available.")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=055e6652ac97e762f53aa4c7d46a2f2a903035e953fc1812351c00200001d87d
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available!


In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q transformers
!pip install -q datasets

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

if 'COLAB_GPU' in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `finetune` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `finetune`


In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                            quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
train_dataset = load_dataset("csv", data_files="dataset.csv", split="train")
train_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 325
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           add_eos_token=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'],padding="max_length", truncation=True)
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/325 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 325
})

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    # rank of the update matrices
    # Lower rank results in smaller matrices with fewer trainable params
    r=8,

    # impacts low-rank approximation aggressiveness
    # increasing value speeds up training
    lora_alpha=64,

    # modules to apply the LoRA update matrices
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "down_proj",
        "up_proj",
        "o_proj"
    ],

    # determines LoRA bias type, influencing training dynamics
    bias="none",

    # regulates model regularization; increasing may lead to underfitting
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
trainer = transformers.Trainer(
    model=model, # Mô hình sử dụng tinh chỉnh
    train_dataset=tokenized_dataset, # Bộ dataset sử dụng tinh chỉnh
    args=transformers.TrainingArguments(
        output_dir="./finetuned-llama-2-model-TuyenSinhPTIT2024",  # Thư mục lưu lại các checkpoint huấn luyện
        per_device_train_batch_size=2, # Số mẫu dữ liệu mỗi bath cho GPU
        gradient_accumulation_steps=2, # Tích lũy gradient sau 2 bước trước khi thực hiện cập nhật trọng số
        num_train_epochs=3, # Số lần duyệt qua toàn bộ dữ liệu huấn luyện
        learning_rate=2e-4, # Xác định mức độ thay đổi trọng số trong mỗi bước cập nhật
        bf16=False,      # Tắt định dạng bfloat16 do phần cứng không hỗ trợ
        optim="paged_adamw_8bit", # Sử dụng tối ưu hóa Adamw phiên bản 8bit để tiết kiệm bộ nhớ và tăng hiệu suất
        logging_dir="./logs",# Thư mục sử dụng để lưu thông tin huấn luyện
        save_strategy="epoch", # Lưu lại model huấn luyện sau mỗi epoch
        logging_steps = 10, # Ghi lại thông tin nhật kí huấn luyện
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False # Tắt cache trong bộ huấn luyện để tối ưu bộ nhớ
trainer.train() # Tiến hành huấn luyện

trainer.save_model("./finetuned_llama2_7b_chat")
tokenizer.save_pretrained("./finetuned_llama2_7b_chat")

In [ ]:

del trainer
import gc
gc.collect()

176

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           trust_remote_code=True)


# modelFinetuned = PeftModel.from_pretrained(base_model,"finetuneModel")
# modelFinetuned = modelFinetuned.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
### ENTER YOUR QUESTION BELOW

question = "Trả lời câu hỏi sau bằng tiếng việt: Điểm chuẩn ngành công nghệ thông tin của Học viện công nghệ bưu chính viễn thông năm 2023 tại cơ sở BVH?"

# Format the question
eval_prompt = f"{question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()
with torch.no_grad():
    print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()

Trả lời câu hỏi sau bằng tiếng việt: Điểm chuẩn ngành công nghệ thông tin của Học viện công nghệ bưu chính viễn thông năm 2023 tại cơ sở BVH?

Điểm chuẩn ngành Công nghệ thông tin của Học viện Công nghệ Bưu chính Viễn thông năm 2023 tại cơ sở ở Hà Nội là 25.50 điểm. Thứ tự nguyện vọng (TTNV) là 1.20 điểm.

Các thứ tự nguyện vọng (TTNV) của các ngành trong cơ sở đã được điều chỉnh theo thứ tự nguyện vọng của ngành dẫn và ngành đi đầu (Sense) để phù hợp với số lượng ngành đăng ký xét tuyển.

Ghi chú: Thứ tự nguyện vọng (TTNV) là một trong các yếu tố xét tuyển của cơ sở giáo dục đào tạo năng lực, cùng với điểm thi tốt nghiệp THPT và điểm ưu tiên (nếu có).


In [ ]:
modelFinetuned = PeftModel.from_pretrained(base_model,"newmodel")
modelFinetuned = modelFinetuned.merge_and_unload()

In [ ]:
modelFinetuned.save_pretrained("./llama-2-chat-hf-for-TuyenSinhPTIT2024")

In [ ]:
modelFinetuned.push_to_hub("llama-2-chat-hf-for-TuyenSinhPTIT2024", use_temp_dir=False)
tokenizer.push_to_hub("llama-2-chat-hf-for-TuyenSinhPTIT2024", use_temp_dir=False)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HuyALT/llama-2-chat-hf-for-TuyenSinhPTIT2024/commit/bc4ddcd7daceecf1e8f3454204b5dcedfbf8da08', commit_message='Upload tokenizer', commit_description='', oid='bc4ddcd7daceecf1e8f3454204b5dcedfbf8da08', pr_url=None, repo_url=RepoUrl('https://huggingface.co/HuyALT/llama-2-chat-hf-for-TuyenSinhPTIT2024', endpoint='https://huggingface.co', repo_type='model', repo_id='HuyALT/llama-2-chat-hf-for-TuyenSinhPTIT2024'), pr_revision=None, pr_num=None)